In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "DD"

# Learnable Root-Based BFS Positional Encoding GCN

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_undirected, degree
import wandb
from collections import deque

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def prep_graph(d):
    d = d.clone()
    if d.edge_index.numel() > 0:
        d.edge_index = to_undirected(d.edge_index, num_nodes=d.num_nodes)
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    deg = degree(d.edge_index[0], d.num_nodes, dtype=torch.float) if d.edge_index.numel() > 0 else torch.zeros(d.num_nodes, dtype=torch.float)
    m = max(deg.max().item(), 1.0)
    deg = (deg / m).unsqueeze(1)
    d.x = torch.cat([d.x, deg], dim=1)
    return d

def bfs_rank_from_root(num_nodes, edge_index_local, root_local):
    adj = [[] for _ in range(num_nodes)]
    u, v = edge_index_local
    for i in range(u.numel()):
        a = int(u[i])
        b = int(v[i])
        if a != b:
            adj[a].append(b)
            adj[b].append(a)
    ranks = torch.full((num_nodes,), -1.0)
    if num_nodes == 0 or root_local < 0 or root_local >= num_nodes:
        return ranks
    q = deque([root_local])
    seen = [False] * num_nodes
    seen[root_local] = True
    order = []
    while q:
        cur = q.popleft()
        order.append(cur)
        for w in adj[cur]:
            if not seen[w]:
                seen[w] = True
                q.append(w)
    L = max(1, len(order))
    for i, node in enumerate(order):
        ranks[node] = float(i) / float(L)
    return ranks

class LearnableBFS_GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, tau=1.0):
        super().__init__()
        self.tau = tau
        self.root_scorer = nn.Sequential(
            nn.Linear(in_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.bfs_proj = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)

    @torch.no_grad()
    def _compute_bfs_rank_batch(self, x, edge_index, batch, root_logits):
        device = x.device
        N = x.size(0)
        ranks = torch.full((N,), -1.0, device='cpu')
        edge_index_cpu = edge_index.detach().cpu()
        batch_cpu = batch.detach().cpu()
        logits_cpu = root_logits.detach().cpu().squeeze(-1)
        num_graphs = int(batch_cpu.max().item()) + 1 if N > 0 else 0
        for g in range(num_graphs):
            idx = (batch_cpu == g).nonzero(as_tuple=False).view(-1)
            n = idx.numel()
            if n == 0:
                continue
            logits_g = logits_cpu[idx]
            probs_g = F.gumbel_softmax(logits_g, tau=self.tau, hard=True)
            root_local = int(probs_g.argmax().item())
            map_gl2lc = torch.full((N,), -1, dtype=torch.long)
            map_gl2lc[idx] = torch.arange(n, dtype=torch.long)
            u_all, v_all = edge_index_cpu
            m_u = map_gl2lc[u_all]
            m_v = map_gl2lc[v_all]
            mask = (m_u >= 0) & (m_v >= 0)
            u_loc = m_u[mask]
            v_loc = m_v[mask]
            ranks_g = bfs_rank_from_root(n, (u_loc, v_loc), root_local)
            ranks[idx] = ranks_g
        return ranks.to(device)

    def forward(self, x, edge_index, batch):
        root_logits = self.root_scorer(x)
        bfs_rank = self._compute_bfs_rank_batch(x, edge_index, batch, root_logits)
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=prep_graph)
    if dataset.num_features == 0:
        for data in dataset:
            data.x = torch.ones((data.num_nodes, 1))
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = LearnableBFS_GCN(in_channels, 64, dataset.num_classes, bfs_dim=8, tau=1.0).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableBFS_STG_seed_{seed}",
        group=f"GCN_{Dataset_name}_LearnableBFS",
        job_type="learnable_bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable BFS",
            "positional_encoding": "Learned root (ST-Gumbel) + BFS rank MLP",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total_samples = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        train_acc = correct / total_samples
        return total_loss / len(train_loader), train_acc

    def test(loader):
        model.eval()
        correct = 0
        total_samples = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        return correct / total_samples

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Seed 1 | Epoch 001 | Loss: 0.6802 | Train Acc: 0.5924 | Test Acc: 0.5678
Seed 1 | Epoch 002 | Loss: 0.6747 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 003 | Loss: 0.6741 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 004 | Loss: 0.6675 | Train Acc: 0.5924 | Test Acc: 0.5678
Seed 1 | Epoch 005 | Loss: 0.6617 | Train Acc: 0.5966 | Test Acc: 0.5805
Seed 1 | Epoch 006 | Loss: 0.6528 | Train Acc: 0.6285 | Test Acc: 0.5720
Seed 1 | Epoch 007 | Loss: 0.6364 | Train Acc: 0.6497 | Test Acc: 0.6483
Seed 1 | Epoch 008 | Loss: 0.6150 | Train Acc: 0.6837 | Test Acc: 0.6525
Seed 1 | Epoch 009 | Loss: 0.5966 | Train Acc: 0.7006 | Test Acc: 0.7076
Seed 1 | Epoch 010 | Loss: 0.5888 | Train Acc: 0.6996 | Test Acc: 0.6356
Seed 1 | Epoch 011 | Loss: 0.5791 | Train Acc: 0.7251 | Test Acc: 0.6822
Seed 1 | Epoch 012 | Loss: 0.5767 | Train Acc: 0.7102 | Test Acc: 0.7076
Seed 1 | Epoch 013 | Loss: 0.5691 | Train Acc: 0.7293 | Test Acc: 0.6822
Seed 1 | Epoch 014 | Loss: 0.5697 | Train Acc: 0.72

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▆▅▄▄▄▄▃▃▃▃▄▃▃▃▃▃▂▂▃▃▃▂▂▃▂▂▃▃▂▂▂▁▁▁▁
test_acc,▁▁▁▁▂▅▅▇▄▆▆▇▆▅▇▆▇▇▆▇▇▆▇▇▇▇▆▇▇██▇▆▇█▇▆▇▇▇
train_acc,▁▁▁▁▁▃▄▅▅▆▆▆▆▇▆▆▆▇▇▇▇▇▆▆▇▆▇▇▇▇▇▆▆███▇▇██
epoch,50
loss,0.50387
test_acc,0.69492
train_acc,0.78238


Seed 2 | Epoch 001 | Loss: 0.6880 | Train Acc: 0.5605 | Test Acc: 0.6144
Seed 2 | Epoch 002 | Loss: 0.6822 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 003 | Loss: 0.6796 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 004 | Loss: 0.6784 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 005 | Loss: 0.6741 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 006 | Loss: 0.6694 | Train Acc: 0.5860 | Test Acc: 0.6144
Seed 2 | Epoch 007 | Loss: 0.6593 | Train Acc: 0.5998 | Test Acc: 0.6356
Seed 2 | Epoch 008 | Loss: 0.6476 | Train Acc: 0.6465 | Test Acc: 0.6356
Seed 2 | Epoch 009 | Loss: 0.6349 | Train Acc: 0.6720 | Test Acc: 0.6992
Seed 2 | Epoch 010 | Loss: 0.6197 | Train Acc: 0.6762 | Test Acc: 0.6653
Seed 2 | Epoch 011 | Loss: 0.6162 | Train Acc: 0.6837 | Test Acc: 0.7500
Seed 2 | Epoch 012 | Loss: 0.6018 | Train Acc: 0.6985 | Test Acc: 0.6992
Seed 2 | Epoch 013 | Loss: 0.5994 | Train Acc: 0.7081 | Test Acc: 0.6992
Seed 2 | Epoch 014 | Loss: 0.5891 | Train Acc: 0.71

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▆▆▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▂▁▂▁▁
test_acc,▁▁▁▁▁▂▅▇▅▅▆▇▆▇▅▅▇▇▇▆▅▇▇▆▇▇▇▇▇█▇▇▇▇▇█▇▇█▇
train_acc,▁▂▂▂▂▄▅▅▅▆▆▇▆▆▇▆▆▇▇▇▆▆▇▇█▇▇████▇█▇▇▆█▆██
epoch,50
loss,0.53322
test_acc,0.76271
train_acc,0.76433


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
loss,█████▇▆▆▅▅▄▄▄▃▄▃▃▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁
test_acc,▁▁▂▁▂▆█▆▆▆█▇███▇▇▇████▇▆█▇█▇▆██▇█▇▇▇█▇▇▇
train_acc,▁▁▁▁▁▂▃▄▅▅▆▆▆▆▇▇▇▇▆▇▇▇▇▆▇▇█▇▇▇▇▇▇▇▇███▇█
epoch,50
loss,0.51449
test_acc,0.70763
train_acc,0.75902


Seed 29 | Epoch 001 | Loss: 0.6801 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 002 | Loss: 0.6775 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 003 | Loss: 0.6746 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 004 | Loss: 0.6708 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 005 | Loss: 0.6654 | Train Acc: 0.5955 | Test Acc: 0.7119
Seed 29 | Epoch 006 | Loss: 0.6626 | Train Acc: 0.6518 | Test Acc: 0.5932
Seed 29 | Epoch 007 | Loss: 0.6481 | Train Acc: 0.6369 | Test Acc: 0.6314
Seed 29 | Epoch 008 | Loss: 0.6350 | Train Acc: 0.6561 | Test Acc: 0.6441
Seed 29 | Epoch 009 | Loss: 0.6204 | Train Acc: 0.6773 | Test Acc: 0.6271
Seed 29 | Epoch 010 | Loss: 0.6123 | Train Acc: 0.6879 | Test Acc: 0.7288
Seed 29 | Epoch 011 | Loss: 0.5974 | Train Acc: 0.7229 | Test Acc: 0.7119
Seed 29 | Epoch 012 | Loss: 0.5839 | Train Acc: 0.7134 | Test Acc: 0.7415
Seed 29 | Epoch 013 | Loss: 0.5866 | Train Acc: 0.7038 | Test Acc: 0.7246
Seed 29 | Epoch 014 | Loss: 0.5805 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▆▅▅▄▄▄▃▄▃▃▃▃▃▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
test_acc,▁▁▁▁▆▃▂▇▆▇▇▇▇▇▇▇▇▇▆▇▆▆▆▇▆▇█▇▇▇▇██▇▇▇█▅█▇
train_acc,▁▁▁▁▁▃▄▅▅▆▆▇▆▆▇▇▅▇▆▇▇▇▆▇▇▇▇█▇▇█▇▇▇▇████▇
epoch,50
loss,0.51514
test_acc,0.72458
train_acc,0.74735


Seed 30 | Epoch 001 | Loss: 0.6850 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 002 | Loss: 0.6797 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 003 | Loss: 0.6801 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 004 | Loss: 0.6732 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 005 | Loss: 0.6722 | Train Acc: 0.6008 | Test Acc: 0.6102
Seed 30 | Epoch 006 | Loss: 0.6629 | Train Acc: 0.5892 | Test Acc: 0.6059
Seed 30 | Epoch 007 | Loss: 0.6524 | Train Acc: 0.6369 | Test Acc: 0.6780
Seed 30 | Epoch 008 | Loss: 0.6368 | Train Acc: 0.6561 | Test Acc: 0.6780
Seed 30 | Epoch 009 | Loss: 0.6244 | Train Acc: 0.6805 | Test Acc: 0.6992
Seed 30 | Epoch 010 | Loss: 0.6018 | Train Acc: 0.6975 | Test Acc: 0.6907
Seed 30 | Epoch 011 | Loss: 0.5980 | Train Acc: 0.6879 | Test Acc: 0.6992
Seed 30 | Epoch 012 | Loss: 0.5910 | Train Acc: 0.7134 | Test Acc: 0.7076
Seed 30 | Epoch 013 | Loss: 0.5782 | Train Acc: 0.7176 | Test Acc: 0.7076
Seed 30 | Epoch 014 | Loss: 0.5757 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
loss,████▇▇▆▆▅▅▄▄▄▃▃▃▃▃▄▃▃▂▂▂▃▂▃▃▂▂▂▂▂▂▂▂▁▂▁▁
test_acc,▁▁▁▁▁▄▄▅▅▅▆▆▇▅▅▅▆▇▇▇▇▆▅▇█▅▇▇▇▇▇▇▆▇▆▇▆▇▇▇
train_acc,▁▁▁▁▂▃▄▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▆▆▇▇▇▇██▇█▇▇█
epoch,50
loss,0.51067
test_acc,0.74576
train_acc,0.77282


# **Layerwise BFS**

In [4]:
# **Layerwise BFS for Protein**
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(1, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels + bfs_dim, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_rank, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        bfs_feat2 = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat2], dim=1)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            root = max(degrees, key=degrees.get) if degrees else 0
            bfs_order = list(nx.bfs_tree(G, root))
            L = max(1, len(bfs_order))
            for i, node in enumerate(bfs_order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        # REDDIT-BINARY has no node features; use degree as feature or 1s
        if not hasattr(data, 'x') or data.x is None:
            deg = torch.tensor([d for _, d in G.degree()], dtype=torch.float).unsqueeze(1)
            data.x = deg
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Layerwise_BFS_{Dataset_name}_seed_{SEED}",
        group=f"GCN_{Dataset_name}_Layerwise_BFS",
        job_type="layerwise-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Layerwise_BFS_Injection",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        acc = correct / len(train_dataset)
        return total_loss / len(train_dataset), acc
    
    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)
    
    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Seed 1 | Epoch 001 | Loss: 0.6845 | Train Acc: 0.5433 | Test Acc: 0.5874
Seed 1 | Epoch 002 | Loss: 0.6781 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 003 | Loss: 0.6764 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 004 | Loss: 0.6753 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 005 | Loss: 0.6722 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 006 | Loss: 0.6701 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 007 | Loss: 0.6658 | Train Acc: 0.5857 | Test Acc: 0.5891
Seed 1 | Epoch 008 | Loss: 0.6617 | Train Acc: 0.6146 | Test Acc: 0.5976
Seed 1 | Epoch 009 | Loss: 0.6558 | Train Acc: 0.6180 | Test Acc: 0.6231
Seed 1 | Epoch 010 | Loss: 0.6452 | Train Acc: 0.6299 | Test Acc: 0.5993
Seed 1 | Epoch 011 | Loss: 0.6362 | Train Acc: 0.6689 | Test Acc: 0.6774
Seed 1 | Epoch 012 | Loss: 0.6287 | Train Acc: 0.6689 | Test Acc: 0.6723
Seed 1 | Epoch 013 | Loss: 0.6272 | Train Acc: 0.6672 | Test Acc: 0.6537
Seed 1 | Epoch 014 | Loss: 0.6315 | Train Acc: 0.67

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▇▇▆▆▅▆▄▅▄▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁
test_acc,▁▁▁▁▁▂▃▂▆▅▅▅█▆▆▅▇█▇█▇▇▇▅▇▇▇▇▆▇█▇▇▇▇▇▇█▇█
train_acc,▁▂▂▂▂▂▃▃▄▅▅▅▆▆▇▆▆▇▇▇▇▇▇▇▆▇▇▇▇██▇▇█▇█▇██▇
epoch,50
loss,0.54177
test_acc,0.71986
train_acc,0.73514


Seed 2 | Epoch 001 | Loss: 0.6818 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 002 | Loss: 0.6807 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 003 | Loss: 0.6797 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 004 | Loss: 0.6799 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 005 | Loss: 0.6788 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 006 | Loss: 0.6748 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 007 | Loss: 0.6740 | Train Acc: 0.5772 | Test Acc: 0.5959
Seed 2 | Epoch 008 | Loss: 0.6722 | Train Acc: 0.5789 | Test Acc: 0.6197
Seed 2 | Epoch 009 | Loss: 0.6715 | Train Acc: 0.6231 | Test Acc: 0.5959
Seed 2 | Epoch 010 | Loss: 0.6651 | Train Acc: 0.5772 | Test Acc: 0.6146
Seed 2 | Epoch 011 | Loss: 0.6615 | Train Acc: 0.6553 | Test Acc: 0.6163
Seed 2 | Epoch 012 | Loss: 0.6497 | Train Acc: 0.6401 | Test Acc: 0.6774
Seed 2 | Epoch 013 | Loss: 0.6405 | Train Acc: 0.6842 | Test Acc: 0.6503
Seed 2 | Epoch 014 | Loss: 0.6315 | Train Acc: 0.67

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███████▇▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁
test_acc,▁▁▁▁▁▁▂▁▂▂▃▆▄▅▅▅▆▆▇▆▆▆▇▇▇▇▆▇▇▇███▇▇▆█▆█▆
train_acc,▁▁▁▁▁▁▃▁▄▄▅▆▆▆▅▇▅▆▆▇▇▇▇█▇▆▇█▇█▇▇██▇▇▇▇▇█
epoch,50
loss,0.53923
test_acc,0.6944
train_acc,0.73684


Seed 3 | Epoch 001 | Loss: 0.6824 | Train Acc: 0.5891 | Test Acc: 0.5840
Seed 3 | Epoch 002 | Loss: 0.6774 | Train Acc: 0.5891 | Test Acc: 0.5840
Seed 3 | Epoch 003 | Loss: 0.6759 | Train Acc: 0.5891 | Test Acc: 0.5840
Seed 3 | Epoch 004 | Loss: 0.6744 | Train Acc: 0.5891 | Test Acc: 0.5840
Seed 3 | Epoch 005 | Loss: 0.6718 | Train Acc: 0.5891 | Test Acc: 0.5840
Seed 3 | Epoch 006 | Loss: 0.6708 | Train Acc: 0.5908 | Test Acc: 0.5840
Seed 3 | Epoch 007 | Loss: 0.6663 | Train Acc: 0.5908 | Test Acc: 0.6299
Seed 3 | Epoch 008 | Loss: 0.6600 | Train Acc: 0.6044 | Test Acc: 0.5840
Seed 3 | Epoch 009 | Loss: 0.6519 | Train Acc: 0.5993 | Test Acc: 0.6401
Seed 3 | Epoch 010 | Loss: 0.6451 | Train Acc: 0.6469 | Test Acc: 0.5976
Seed 3 | Epoch 011 | Loss: 0.6416 | Train Acc: 0.6604 | Test Acc: 0.6095
Seed 3 | Epoch 012 | Loss: 0.6418 | Train Acc: 0.6435 | Test Acc: 0.6316
Seed 3 | Epoch 013 | Loss: 0.6352 | Train Acc: 0.6570 | Test Acc: 0.7097
Seed 3 | Epoch 014 | Loss: 0.6116 | Train Acc: 0.71

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██████▇▇▇▆▅▄▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▂
test_acc,▁▁▁▁▁▁▂▆▅▅▇▁▆▂██▆▇▇▇▇▇██▇▇▇▇▇▇▇▇▇█▆███▇█
train_acc,▁▁▁▁▁▁▂▃▄▅▆▄▅▆▇▇▇▇▇▇▇▇▇▇█▇▇▇██▇███▆█████
epoch,50
loss,0.51264
test_acc,0.691
train_acc,0.77419


Seed 29 | Epoch 001 | Loss: 0.6795 | Train Acc: 0.5603 | Test Acc: 0.5789
Seed 29 | Epoch 002 | Loss: 0.6803 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 003 | Loss: 0.6758 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 004 | Loss: 0.6754 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 005 | Loss: 0.6734 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 006 | Loss: 0.6738 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 007 | Loss: 0.6723 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 008 | Loss: 0.6729 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 009 | Loss: 0.6672 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 010 | Loss: 0.6634 | Train Acc: 0.5942 | Test Acc: 0.5806
Seed 29 | Epoch 011 | Loss: 0.6594 | Train Acc: 0.5959 | Test Acc: 0.5840
Seed 29 | Epoch 012 | Loss: 0.6529 | Train Acc: 0.5993 | Test Acc: 0.5840
Seed 29 | Epoch 013 | Loss: 0.6494 | Train Acc: 0.6197 | Test Acc: 0.6163
Seed 29 | Epoch 014 | Loss: 0.6448 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███████▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▂▃▂▃▄▃▂▂▂▂▂▂▁▂▂▁▁
test_acc,▁▁▁▁▁▁▁▁▁▁▃▂▃▇▃█▆▇▅▇▆███▄█▇█▇█▇▇▇██▆▇▄▇█
train_acc,▁▂▂▂▂▂▂▂▂▂▄▅▅▅▅▆▆▆▇▆▇▇▆▇▇▇▇█▆▆▇▇▇▇▇▇▇▇▇█
epoch,50
loss,0.54066
test_acc,0.72496
train_acc,0.76231


Seed 30 | Epoch 001 | Loss: 0.6838 | Train Acc: 0.5450 | Test Acc: 0.5993
Seed 30 | Epoch 002 | Loss: 0.6835 | Train Acc: 0.5739 | Test Acc: 0.5993
Seed 30 | Epoch 003 | Loss: 0.6796 | Train Acc: 0.5739 | Test Acc: 0.5993
Seed 30 | Epoch 004 | Loss: 0.6794 | Train Acc: 0.5739 | Test Acc: 0.5993
Seed 30 | Epoch 005 | Loss: 0.6772 | Train Acc: 0.5739 | Test Acc: 0.5993
Seed 30 | Epoch 006 | Loss: 0.6740 | Train Acc: 0.5739 | Test Acc: 0.6027
Seed 30 | Epoch 007 | Loss: 0.6711 | Train Acc: 0.5772 | Test Acc: 0.6010
Seed 30 | Epoch 008 | Loss: 0.6636 | Train Acc: 0.5789 | Test Acc: 0.6486
Seed 30 | Epoch 009 | Loss: 0.6617 | Train Acc: 0.6265 | Test Acc: 0.6231
Seed 30 | Epoch 010 | Loss: 0.6558 | Train Acc: 0.6299 | Test Acc: 0.6333
Seed 30 | Epoch 011 | Loss: 0.6415 | Train Acc: 0.6350 | Test Acc: 0.6503
Seed 30 | Epoch 012 | Loss: 0.6429 | Train Acc: 0.6587 | Test Acc: 0.6774
Seed 30 | Epoch 013 | Loss: 0.6246 | Train Acc: 0.6927 | Test Acc: 0.6791
Seed 30 | Epoch 014 | Loss: 0.6142 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▇▇▆▆▅▅▄▄▄▄▄▃▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
test_acc,▁▁▁▁▁▁▄▂▄▅▁▆▆▆▆▅▆▇▆▅▅▆▆▇▄▇▇▇▇▇▇█████████
train_acc,▁▁▁▁▁▁▃▃▃▄▅▄▆▆▆▆▆▇▆▅▆▆▇▇▇▇▇█▇▇▇▇▇█▇▇▇███
epoch,50
loss,0.53471
test_acc,0.72666
train_acc,0.75722


# **GCN Learnable BFS Sinusoidal**

In [4]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
import math
from torch import nn

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class LearnableBFSPositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=512):
        super().__init__()
        self.lin = nn.Linear(1, dim)
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)
        self.max_len = max_len
    def forward(self, bfs_rank):
        idx = (bfs_rank.clamp(0, 1) * (self.max_len - 1)).long()
        return self.lin(bfs_rank.unsqueeze(1)) + self.pe[idx]

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=32, max_len=512):
        super().__init__()
        self.bfs_enc = LearnableBFSPositionalEncoding(bfs_dim, max_len)
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, bfs_rank, edge_index, batch):
        x = torch.cat([x, self.bfs_enc(bfs_rank)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            deg = dict(G.degree())
            root = max(deg, key=deg.get) if len(deg) > 0 else 0
            order = list(nx.bfs_tree(G, root))
            L = max(1, len(order))
            for i, node in enumerate(order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        return data
    return transform
    
def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)  # 80% for train
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableSinBFS_NoLayerwise_seed_{SEED}",
        group=f"GCN_{Dataset_name}_LearnableSinBFS",
        job_type="learnable-sin-bfs-nolayerwise",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable_Sinusoidal_BFS",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        bfs_dim=32,
        max_len=512
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Seed 1 | Epoch 001 | Loss: 0.6788 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 002 | Loss: 0.6756 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 003 | Loss: 0.6717 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 004 | Loss: 0.6766 | Train Acc: 0.5924 | Test Acc: 0.5678
Seed 1 | Epoch 005 | Loss: 0.6678 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 006 | Loss: 0.6622 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 007 | Loss: 0.6535 | Train Acc: 0.5977 | Test Acc: 0.5890
Seed 1 | Epoch 008 | Loss: 0.6470 | Train Acc: 0.6306 | Test Acc: 0.5720
Seed 1 | Epoch 009 | Loss: 0.6429 | Train Acc: 0.6507 | Test Acc: 0.6525
Seed 1 | Epoch 010 | Loss: 0.6237 | Train Acc: 0.6847 | Test Acc: 0.5763
Seed 1 | Epoch 011 | Loss: 0.6157 | Train Acc: 0.6826 | Test Acc: 0.5805
Seed 1 | Epoch 012 | Loss: 0.5842 | Train Acc: 0.7102 | Test Acc: 0.7161
Seed 1 | Epoch 013 | Loss: 0.5758 | Train Acc: 0.7197 | Test Acc: 0.7246
Seed 1 | Epoch 014 | Loss: 0.5559 | Train Acc: 0.72

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▇▇▆▆▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▃▃▁▁▁▁▁▁▁
test_acc,▁▁▁▁▁▂▁▄▁▁▃▇▆▆▇▆▇▇▇▇▇▃▇▇▇██▇█▇▆▆█▇███▇██
train_acc,▁▁▁▁▁▁▂▃▄▄▅▆▅▆▆▆▇▇▇▇▇▆▇▇▇▇▇▇▇▇▆▇▇▇█▇▇██▇
epoch,50
loss,0.45227
test_acc,0.76695
train_acc,0.78662


Seed 2 | Epoch 001 | Loss: 0.6865 | Train Acc: 0.5626 | Test Acc: 0.6144
Seed 2 | Epoch 002 | Loss: 0.6802 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 003 | Loss: 0.6771 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 004 | Loss: 0.6735 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 005 | Loss: 0.6719 | Train Acc: 0.5796 | Test Acc: 0.6144
Seed 2 | Epoch 006 | Loss: 0.6718 | Train Acc: 0.5902 | Test Acc: 0.6441
Seed 2 | Epoch 007 | Loss: 0.6652 | Train Acc: 0.6157 | Test Acc: 0.6144
Seed 2 | Epoch 008 | Loss: 0.6603 | Train Acc: 0.5977 | Test Acc: 0.6144
Seed 2 | Epoch 009 | Loss: 0.6537 | Train Acc: 0.6221 | Test Acc: 0.6737
Seed 2 | Epoch 010 | Loss: 0.6372 | Train Acc: 0.6369 | Test Acc: 0.7161
Seed 2 | Epoch 011 | Loss: 0.6242 | Train Acc: 0.6762 | Test Acc: 0.6695
Seed 2 | Epoch 012 | Loss: 0.6215 | Train Acc: 0.6656 | Test Acc: 0.6610
Seed 2 | Epoch 013 | Loss: 0.6062 | Train Acc: 0.7081 | Test Acc: 0.6992
Seed 2 | Epoch 014 | Loss: 0.5887 | Train Acc: 0.69

epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▇▇▆▆▅▄▄▅▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁▁▁▁▁▁▃▅▃▃▇█▇▇▆▇▅▇▇▅█▇▇▇▆█▇█▆▆█▆▇██▇███▅
train_acc,▁▂▂▂▂▃▂▃▄▅▆▆▅▇▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇████▇▇███
epoch,50
loss,0.47368
test_acc,0.70763
train_acc,0.78238


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,████▇▇▇▅▅▄▃▃▃▃▃▃▂▂▃▂▂▂▃▃▂▂▂▂▂▂▁▁▃▂▁▁▁▂▂▁
test_acc,▄▄▄▇▄▇▆▇▁██▇▆▆█████▆██▇▆▆▇███▆▇▆█▇█▇▇▇▇▇
train_acc,▁▁▁▁▁▂▃▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇███▇▇█▇█▇▇█
epoch,50
loss,0.44496
test_acc,0.75424
train_acc,0.79299


Seed 29 | Epoch 001 | Loss: 0.6854 | Train Acc: 0.5764 | Test Acc: 0.5932
Seed 29 | Epoch 002 | Loss: 0.6771 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 003 | Loss: 0.6753 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 004 | Loss: 0.6720 | Train Acc: 0.5860 | Test Acc: 0.5975
Seed 29 | Epoch 005 | Loss: 0.6726 | Train Acc: 0.5945 | Test Acc: 0.6102
Seed 29 | Epoch 006 | Loss: 0.6654 | Train Acc: 0.5860 | Test Acc: 0.5932
Seed 29 | Epoch 007 | Loss: 0.6640 | Train Acc: 0.5955 | Test Acc: 0.5932
Seed 29 | Epoch 008 | Loss: 0.6583 | Train Acc: 0.6168 | Test Acc: 0.5932
Seed 29 | Epoch 009 | Loss: 0.6592 | Train Acc: 0.6231 | Test Acc: 0.7373
Seed 29 | Epoch 010 | Loss: 0.6409 | Train Acc: 0.6423 | Test Acc: 0.6483
Seed 29 | Epoch 011 | Loss: 0.6263 | Train Acc: 0.6688 | Test Acc: 0.6229
Seed 29 | Epoch 012 | Loss: 0.6167 | Train Acc: 0.6868 | Test Acc: 0.7076
Seed 29 | Epoch 013 | Loss: 0.6063 | Train Acc: 0.6921 | Test Acc: 0.7458
Seed 29 | Epoch 014 | Loss: 0.6040 | T

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▇▇▆▆▆▅▄▄▃▃▃▃▃▂▃▃▃▃▃▃▂▂▃▂▂▂▂▁▁▁▂▁▂▁
test_acc,▁▁▁▁▂▁▁▆▃▂▆▃▅▅▇▅█▇▇▇▇▇█▇▇▇███▇██▇███▇▇▅▆
train_acc,▁▁▁▂▁▂▃▃▄▅▆▆▆▇▆▇▆▇▇▇▇▇▇▇▇▇█▇▇███████████
epoch,50
loss,0.46249
test_acc,0.74576
train_acc,0.78238


Seed 30 | Epoch 001 | Loss: 0.6811 | Train Acc: 0.5722 | Test Acc: 0.6017
Seed 30 | Epoch 002 | Loss: 0.6841 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 003 | Loss: 0.6774 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 004 | Loss: 0.6746 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 005 | Loss: 0.6741 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 006 | Loss: 0.6710 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 007 | Loss: 0.6677 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 008 | Loss: 0.6668 | Train Acc: 0.5828 | Test Acc: 0.6017
Seed 30 | Epoch 009 | Loss: 0.6588 | Train Acc: 0.5849 | Test Acc: 0.6017
Seed 30 | Epoch 010 | Loss: 0.6539 | Train Acc: 0.5987 | Test Acc: 0.7246
Seed 30 | Epoch 011 | Loss: 0.6398 | Train Acc: 0.6561 | Test Acc: 0.6525
Seed 30 | Epoch 012 | Loss: 0.6198 | Train Acc: 0.6932 | Test Acc: 0.6525
Seed 30 | Epoch 013 | Loss: 0.6055 | Train Acc: 0.6964 | Test Acc: 0.6483
Seed 30 | Epoch 014 | Loss: 0.5925 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▇▇▇▇▅▅▄▄▄▃▃▃▃▃▂▂▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▃
test_acc,▁▁▁▁▁▁▁▁▅▃▃▆▄▆▇▇▆▇▇▆▆▇▆▆▆▇▅▆▆▆▇▇▇▇▇█▇▇█▆
train_acc,▁▁▁▁▁▁▁▁▂▄▅▅▆▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇███████▇█▇█▇
epoch,50
loss,0.52108
test_acc,0.73305
train_acc,0.75053


# **BFS and centrality**

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = nn.Linear(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent2, edge_index, batch):
        if x is None:
            x = torch.ones((cent2.shape[0], 1), device=cent2.device)
        x = torch.cat([x, self.cent_enc(cent2.unsqueeze(1))], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_centrality_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        n = data.num_nodes
        deg_cent = nx.degree_centrality(G) if G.number_of_nodes() > 0 else {}
        cent = torch.zeros((n,), dtype=torch.float)
        for u in range(n):
            cent[u] = float(deg_cent.get(u, 0.0))
        data.cent2 = cent
        if getattr(data, "x", None) is None:
            data.x = torch.ones((n, 1), dtype=torch.float)
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_centrality_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_CentralityOnly_seed_{SEED}",
        group=f"GCN_{Dataset_name}_CentralityOnly",
        job_type="centrality-only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Centrality_Only",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features if dataset.num_features else 1,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        cent_dim=16
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Seed 1 | Epoch 001 | Loss: 0.6776 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 002 | Loss: 0.6734 | Train Acc: 0.5913 | Test Acc: 0.5678
Seed 1 | Epoch 003 | Loss: 0.6692 | Train Acc: 0.5977 | Test Acc: 0.5720
Seed 1 | Epoch 004 | Loss: 0.6655 | Train Acc: 0.5945 | Test Acc: 0.5890
Seed 1 | Epoch 005 | Loss: 0.6562 | Train Acc: 0.6125 | Test Acc: 0.5720
Seed 1 | Epoch 006 | Loss: 0.6392 | Train Acc: 0.6433 | Test Acc: 0.6102
Seed 1 | Epoch 007 | Loss: 0.6115 | Train Acc: 0.6837 | Test Acc: 0.6144
Seed 1 | Epoch 008 | Loss: 0.5834 | Train Acc: 0.7113 | Test Acc: 0.6186
Seed 1 | Epoch 009 | Loss: 0.5536 | Train Acc: 0.7282 | Test Acc: 0.6822
Seed 1 | Epoch 010 | Loss: 0.5396 | Train Acc: 0.7452 | Test Acc: 0.6568
Seed 1 | Epoch 011 | Loss: 0.5214 | Train Acc: 0.7473 | Test Acc: 0.6907
Seed 1 | Epoch 012 | Loss: 0.5070 | Train Acc: 0.7760 | Test Acc: 0.7203
Seed 1 | Epoch 013 | Loss: 0.5043 | Train Acc: 0.7654 | Test Acc: 0.7712
Seed 1 | Epoch 014 | Loss: 0.5151 | Train Acc: 0.76

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▆▅▄▄▃▂▃▂▂▂▃▂▂▂▂▁▂▁▂▁▂▁▂▂▁▁▁▂▂▁▁▁▁▁▁
test_acc,▁▁▁▂▁▃▃▅▄▅█▆▇█▆▆█▇▇▅████▅█████▆█▇▆██▆███
train_acc,▁▁▁▁▂▄▅▆▆▆▇▇▇▇▇▇▇██▇▇▇▇██▇███▇▇████████▇
epoch,50
loss,0.47021
test_acc,0.77119
train_acc,0.78556


Seed 2 | Epoch 001 | Loss: 0.6800 | Train Acc: 0.5690 | Test Acc: 0.6144
Seed 2 | Epoch 002 | Loss: 0.6703 | Train Acc: 0.5902 | Test Acc: 0.7331
Seed 2 | Epoch 003 | Loss: 0.6639 | Train Acc: 0.6136 | Test Acc: 0.6356
Seed 2 | Epoch 004 | Loss: 0.6433 | Train Acc: 0.6476 | Test Acc: 0.6356
Seed 2 | Epoch 005 | Loss: 0.6411 | Train Acc: 0.6263 | Test Acc: 0.6568
Seed 2 | Epoch 006 | Loss: 0.6032 | Train Acc: 0.6783 | Test Acc: 0.7119
Seed 2 | Epoch 007 | Loss: 0.5811 | Train Acc: 0.7070 | Test Acc: 0.7331
Seed 2 | Epoch 008 | Loss: 0.5468 | Train Acc: 0.7505 | Test Acc: 0.7797
Seed 2 | Epoch 009 | Loss: 0.5200 | Train Acc: 0.7696 | Test Acc: 0.7373
Seed 2 | Epoch 010 | Loss: 0.5156 | Train Acc: 0.7548 | Test Acc: 0.7627
Seed 2 | Epoch 011 | Loss: 0.5239 | Train Acc: 0.7505 | Test Acc: 0.7627
Seed 2 | Epoch 012 | Loss: 0.5112 | Train Acc: 0.7611 | Test Acc: 0.7797
Seed 2 | Epoch 013 | Loss: 0.5054 | Train Acc: 0.7728 | Test Acc: 0.7669
Seed 2 | Epoch 014 | Loss: 0.5055 | Train Acc: 0.77

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▅▃▃▂▃▂▂▂▂▂▂▁▃▁▂▁▁▁▂▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁
test_acc,▁▆▂▂▃▆▇▆▇▇▇▇▇▇▆▆▇█▇▇▇█▇██▇██▇▇▇▇█▇██▇▇▇▇
train_acc,▁▂▂▃▃▅▇▇▇▇▇▇▇▇▇▇█▇█▇██████████▇█████████
epoch,50
loss,0.47475
test_acc,0.77966
train_acc,0.78344


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
loss,█████▇▆▅▅▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▂▂▂▁▁▁▁▁▁
test_acc,▁▁▁▁▁▃▇▅▇▇▆▇▆▆▇▇▆▇▇▇▇▇▆▇▆▇▅▆▇▆█▇▇▇▇█▇▇▇█
train_acc,▁▁▁▁▂▃▄▅▄▇▇▆▇▇▇▇█▇▇█▇███▇███▇▇█████▇▇▇██
epoch,50
loss,0.4724
test_acc,0.82203
train_acc,0.77919


Seed 29 | Epoch 001 | Loss: 0.6830 | Train Acc: 0.5764 | Test Acc: 0.5932
Seed 29 | Epoch 002 | Loss: 0.6764 | Train Acc: 0.5849 | Test Acc: 0.5932
Seed 29 | Epoch 003 | Loss: 0.6732 | Train Acc: 0.5860 | Test Acc: 0.5932
Seed 29 | Epoch 004 | Loss: 0.6657 | Train Acc: 0.5913 | Test Acc: 0.5975
Seed 29 | Epoch 005 | Loss: 0.6569 | Train Acc: 0.6008 | Test Acc: 0.6441
Seed 29 | Epoch 006 | Loss: 0.6434 | Train Acc: 0.6656 | Test Acc: 0.6992
Seed 29 | Epoch 007 | Loss: 0.6219 | Train Acc: 0.6868 | Test Acc: 0.6949
Seed 29 | Epoch 008 | Loss: 0.5931 | Train Acc: 0.7028 | Test Acc: 0.6568
Seed 29 | Epoch 009 | Loss: 0.5795 | Train Acc: 0.7028 | Test Acc: 0.7288
Seed 29 | Epoch 010 | Loss: 0.5438 | Train Acc: 0.7463 | Test Acc: 0.7500
Seed 29 | Epoch 011 | Loss: 0.5431 | Train Acc: 0.7473 | Test Acc: 0.7458
Seed 29 | Epoch 012 | Loss: 0.5241 | Train Acc: 0.7580 | Test Acc: 0.7627
Seed 29 | Epoch 013 | Loss: 0.5098 | Train Acc: 0.7739 | Test Acc: 0.7288
Seed 29 | Epoch 014 | Loss: 0.5445 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▆▅▃▃▃▂▂▂▂▂▂▃▂▂▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
test_acc,▁▁▁▁▃▅▃▆▇▇▆▇▇▇▇████▇▇█▆▇▇██████████████▇
train_acc,▁▁▁▁▂▄▅▅▆▆▇▆▇▇▇▇█▇█▇██▇█████▇██▇████▇███
epoch,50
loss,0.47584
test_acc,0.76695
train_acc,0.79724


Seed 30 | Epoch 001 | Loss: 0.6804 | Train Acc: 0.5594 | Test Acc: 0.6017
Seed 30 | Epoch 002 | Loss: 0.6724 | Train Acc: 0.5828 | Test Acc: 0.6059
Seed 30 | Epoch 003 | Loss: 0.6702 | Train Acc: 0.5977 | Test Acc: 0.6017
Seed 30 | Epoch 004 | Loss: 0.6615 | Train Acc: 0.6062 | Test Acc: 0.6144
Seed 30 | Epoch 005 | Loss: 0.6514 | Train Acc: 0.6072 | Test Acc: 0.6441
Seed 30 | Epoch 006 | Loss: 0.6327 | Train Acc: 0.6699 | Test Acc: 0.7373
Seed 30 | Epoch 007 | Loss: 0.6082 | Train Acc: 0.6794 | Test Acc: 0.6822
Seed 30 | Epoch 008 | Loss: 0.5668 | Train Acc: 0.7272 | Test Acc: 0.7203
Seed 30 | Epoch 009 | Loss: 0.5419 | Train Acc: 0.7420 | Test Acc: 0.7712
Seed 30 | Epoch 010 | Loss: 0.5328 | Train Acc: 0.7473 | Test Acc: 0.7203
Seed 30 | Epoch 011 | Loss: 0.5353 | Train Acc: 0.7399 | Test Acc: 0.7458
Seed 30 | Epoch 012 | Loss: 0.5225 | Train Acc: 0.7473 | Test Acc: 0.7246
Seed 30 | Epoch 013 | Loss: 0.5264 | Train Acc: 0.7431 | Test Acc: 0.7669
Seed 30 | Epoch 014 | Loss: 0.5088 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
loss,███▇▇▆▄▃▃▃▃▂▂▂▂▂▁▂▂▂▁▁▂▂▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁
test_acc,▁▁▁▁▃▄▅▇▅▆▇▇▇█▇█▇█████████▇▇██▇███▇███▇█
train_acc,▁▂▂▂▄▆▆▇▆▇▇▇▇▇██▇▇▇██▇▇████▇▇███████████
epoch,50
loss,0.47676
test_acc,0.79237
train_acc,0.78556
